In [31]:
import pandas as pd

df = pd.read_csv(r'C:\Users\DELL\Desktop\Electronics\data\Customers.csv',encoding='utf-8',encoding_errors='ignore')
df

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,7/3/1939
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,9/27/1979
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,5/26/1947
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,9/17/1957
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,11/19/1965
...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Dukov,Houston,TX,Texas,77017,United States,North America,3/25/1936
15262,2099618,Male,Justin Solrzano,Mclean,VA,Virginia,22101,United States,North America,2/16/1992
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,11/9/1937
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,10/12/1937


In [32]:
df

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,7/3/1939
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,9/27/1979
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,5/26/1947
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,9/17/1957
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,11/19/1965
...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Dukov,Houston,TX,Texas,77017,United States,North America,3/25/1936
15262,2099618,Male,Justin Solrzano,Mclean,VA,Virginia,22101,United States,North America,2/16/1992
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,11/9/1937
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,10/12/1937


# EDA: Data Cleaning and Preprocessing

In [33]:
%pip install unidecode

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15266 entries, 0 to 15265
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerKey  15266 non-null  int64 
 1   Gender       15266 non-null  object
 2   Name         15266 non-null  object
 3   City         15266 non-null  object
 4   State Code   15256 non-null  object
 5   State        15266 non-null  object
 6   Zip Code     15266 non-null  object
 7   Country      15266 non-null  object
 8   Continent    15266 non-null  object
 9   Birthday     15266 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.2+ MB


In [35]:
#Check for Missing Values
df.isnull().sum()

CustomerKey     0
Gender          0
Name            0
City            0
State Code     10
State           0
Zip Code        0
Country         0
Continent       0
Birthday        0
dtype: int64

In [36]:
#Gender
df['Gender'] = df['Gender'].str.upper()

In [37]:

import re
from unidecode import unidecode

#Name
df['Name'] = df['Name'].apply(lambda x: re.sub(r'[^A-Za-z0-9\s]', '', str(x)))

In [38]:
#City
df['City'] = df['City'].str.strip().str.title()

In [39]:
import re

def clean_state_code(value):
    if not isinstance(value, str):  # Check if the value is not a string
        return ''  #return default values
    
    #extract 2-letter uppercase
    state_abbr = re.findall(r'\b[A-Za-z]{2}\b', value)  #match 2 consecutive letters
    
    if state_abbr:  # If they had a 2-letter abbreviation
        return state_abbr[0].upper()  # Returns and convert to uppercase
    else:
        # If there is no match,then they return the first two letters from the string
        return unidecode(value[:2]).upper()
    
#cleaning function to the 'State Code'
df['State Code'] = df['State Code'].apply(clean_state_code).astype(str)

In [40]:
#State
df['State'] = df['State'].str.strip().str.title()

In [41]:
import re

# Zip Code:It remove non-numeric characters and maintain them as string
df['Zip Code'] = df['Zip Code'].apply(lambda x: re.sub(r'[^0-9]', '', str(x)).zfill(5) if pd.notnull(x) else x)

In [42]:
#Continent
df['Continent'] = df['Continent'].str.strip().str.title()

In [43]:
#Birthday
df['Birthday'] = pd.to_datetime(df['Birthday'], errors='coerce')  #'coerce' turns invalid dates to NaT

In [44]:
#Duplicatation handling
df = df.drop_duplicates()

In [45]:
#Demographic Distribution(Age calculation)
df['Age'] = (pd.to_datetime('today') - df['Birthday']).dt.days // 365

In [46]:
#customers based on their age groups
bins = [0, 18, 30, 40, 50, 60, 100]
labels = ['<18', '18-30', '30-40', '40-50', '50-60', '60+']
df['Age Group'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)
df['Age Group'] = df['Age Group'].astype(str)

In [47]:
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15266 entries, 0 to 15265
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   CustomerKey  15266 non-null  int64         
 1   Gender       15266 non-null  object        
 2   Name         15266 non-null  object        
 3   City         15266 non-null  object        
 4   State Code   15266 non-null  object        
 5   State        15266 non-null  object        
 6   Zip Code     15266 non-null  object        
 7   Country      15266 non-null  object        
 8   Continent    15266 non-null  object        
 9   Birthday     15266 non-null  datetime64[ns]
 10  Age          15266 non-null  int64         
 11  Age Group    15266 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 1.4+ MB


,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday,Age,Age Group
0,301,FEMALE,Lilly Harding,Wandearah East,SA,South Australia,05523,Australia,Australia,1939-07-03,85,60+
1,325,FEMALE,Madison Hull,Mount Budd,WA,Western Australia,06522,Australia,Australia,1979-09-27,45,40-50
2,554,FEMALE,Claire Ferres,Winjallok,VI,Victoria,03380,Australia,Australia,1947-05-26,77,60+
3,786,MALE,Jai Poltpalingada,Middle River,SA,South Australia,05223,Australia,Australia,1957-09-17,67,60+
4,1042,MALE,Aidan Pankhurst,Tawonga South,VI,Victoria,03698,Australia,Australia,1965-11-19,59,50-60
...,...,...,...,...,...,...,...,...,...,...,...,...
15261,2099600,FEMALE,Denisa Dukov,Houston,TX,Texas,77017,United States,North America,1936-03-25,88,60+
15262,2099618,MALE,Justin Solrzano,Mclean,VA,Virginia,22101,United States,North America,1992-02-16,32,30-40
15263,2099758,MALE,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,1937-11-09,87,60+
15264,2099862,FEMALE,Lorenza Rush,Riverside,CA,California,92501,United States,North America,1937-10-12,87,60+


In [48]:
df.isnull().sum()

CustomerKey    0
Gender         0
Name           0
City           0
State Code     0
State          0
Zip Code       0
Country        0
Continent      0
Birthday       0
Age            0
Age Group      0
dtype: int64

In [49]:
df.head(5)

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday,Age,Age Group
0,301,FEMALE,Lilly Harding,Wandearah East,SA,South Australia,05523,Australia,Australia,1939-07-03,85,60+
1,325,FEMALE,Madison Hull,Mount Budd,WA,Western Australia,06522,Australia,Australia,1979-09-27,45,40-50
2,554,FEMALE,Claire Ferres,Winjallok,VI,Victoria,03380,Australia,Australia,1947-05-26,77,60+
3,786,MALE,Jai Poltpalingada,Middle River,SA,South Australia,05223,Australia,Australia,1957-09-17,67,60+
4,1042,MALE,Aidan Pankhurst,Tawonga South,VI,Victoria,03698,Australia,Australia,1965-11-19,59,50-60


In [50]:
df.columns

Index(['CustomerKey', 'Gender', 'Name', 'City', 'State Code', 'State',
       'Zip Code', 'Country', 'Continent', 'Birthday', 'Age', 'Age Group'],
      dtype='object')

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15266 entries, 0 to 15265
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   CustomerKey  15266 non-null  int64         
 1   Gender       15266 non-null  object        
 2   Name         15266 non-null  object        
 3   City         15266 non-null  object        
 4   State Code   15266 non-null  object        
 5   State        15266 non-null  object        
 6   Zip Code     15266 non-null  object        
 7   Country      15266 non-null  object        
 8   Continent    15266 non-null  object        
 9   Birthday     15266 non-null  datetime64[ns]
 10  Age          15266 non-null  int64         
 11  Age Group    15266 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 1.4+ MB


In [52]:
df.to_csv("customers.csv")

In [53]:
df.isna().sum()

CustomerKey    0
Gender         0
Name           0
City           0
State Code     0
State          0
Zip Code       0
Country        0
Continent      0
Birthday       0
Age            0
Age Group      0
dtype: int64

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15266 entries, 0 to 15265
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   CustomerKey  15266 non-null  int64         
 1   Gender       15266 non-null  object        
 2   Name         15266 non-null  object        
 3   City         15266 non-null  object        
 4   State Code   15266 non-null  object        
 5   State        15266 non-null  object        
 6   Zip Code     15266 non-null  object        
 7   Country      15266 non-null  object        
 8   Continent    15266 non-null  object        
 9   Birthday     15266 non-null  datetime64[ns]
 10  Age          15266 non-null  int64         
 11  Age Group    15266 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 1.4+ MB


In [55]:
df.isna().sum()

CustomerKey    0
Gender         0
Name           0
City           0
State Code     0
State          0
Zip Code       0
Country        0
Continent      0
Birthday       0
Age            0
Age Group      0
dtype: int64

In [56]:
df.columns

Index(['CustomerKey', 'Gender', 'Name', 'City', 'State Code', 'State',
       'Zip Code', 'Country', 'Continent', 'Birthday', 'Age', 'Age Group'],
      dtype='object')

In [57]:
import mysql.connector
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123456789",   
)
cursor = connection.cursor()
cursor.execute("CREATE DATABASE Electronics")
print("Database Created successfully")

Database Created successfully


In [58]:
import mysql.connector
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123456789",
    database="Electronics" 
)
cursor = connection.cursor()
cursor.execute("""CREATE TABLE customer (
    CustomerKey INT NOT NULL,
    Gender VARCHAR(255) NOT NULL,
    Name VARCHAR(255) NOT NULL,
    City VARCHAR(255) NOT NULL,
    State_Code VARCHAR(255) NOT NULL,
    State VARCHAR(255) NOT NULL,
    Zip_Code VARCHAR(255) NOT NULL,
    Country VARCHAR(255) NOT NULL,
    Continent VARCHAR(255) NOT NULL,
    Birthday DATE NOT NULL,
    Age INT NOT NULL,
    Age_Group VARCHAR(255) NOT NULL
)""")
print("Table Created successfully")

Table Created successfully


In [59]:
insert_query = ("""INSERT INTO customer (CustomerKey,Gender,Name,City,State_Code,State,Zip_Code,Country,Continent,Birthday,Age,Age_Group) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
""")
data =df.values.tolist()
try:
    cursor.executemany(insert_query, data)
    connection.commit()
    print("Values inserted successfully")
except Exception as e:
    print(f"Error: {e}")

Values inserted successfully


In [60]:
df.isna().sum()

CustomerKey    0
Gender         0
Name           0
City           0
State Code     0
State          0
Zip Code       0
Country        0
Continent      0
Birthday       0
Age            0
Age Group      0
dtype: int64